# 2 - Pipeline for measure the metrics for the hard queries generated

In [1]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.2 MB/s eta 0:00:00


In [2]:
import os, json, re, torch
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
from google.colab import userdata
from datasets import load_dataset
import pathlib
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=['datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet'],
    token=hf_token,
)
# setup hugging faces
hf_api = HfApi(token=hf_token)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

datasets/procesed/hard_negative_queries/(…):   0%|          | 0.00/20.3k [00:00<?, ?B/s]

# 2 - Functions

In [4]:
!pip install outlines

In [31]:
path_procesed_hnq = pathlib.Path('/content/datasets/contraint_lists_for_generated_queries')

if not pathlib.Path(path_procesed_hnq).exists():
  pathlib.Path.mkdir(path_procesed_hnq,exist_ok=True,parents=True)

def generate_constraint_list(df_queries,
                             model,
                             tokenizer,
                             system_template_path=None,
                             user_template_path=None,
                             save_local=True):
  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i,batch in enumerate(batches):
    # Create the parquet dataset
    new_rows = []
    for row in batch.to_pylist():
      query = "-".join(row['hard_negative_query'])

      for _ in range(3):
        messages = [
            {"role": "system", "content": system_template()},
            {"role": "user", "content": user_template(user_query=query)}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        ## Append new column for table
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)
        if len(response)>=0:
          new_rows.append(response)



    batch_pylist = batch.to_pylist()
    new_data = []
    for new_row,data in zip(new_rows,batch_pylist):
        data["constraint_list"] = new_row
        new_data.append(data)

    if save_local:
      batch = pa.Table.from_pylist(new_data)
      pq.write_table(batch, f"/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_{i}.parquet")
      print(f"Batch {i} done.")

def check_constraint_in_lists(row_review,row_contraints):
      list_negative_in_reviews = row_review["negative_points"]
      list_contraints = row_contraints["constraint_list"]
      if list_negative_in_reviews in list_contraints:
        return 1
      else:
        return 0

def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


1 - Calculate similarity scores

2 - Calculate the Constraint Preservation Rate

In [6]:
df_reviews = pq.read_table('/content/datasets/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet')

In [18]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_query_restritions_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """
    You are a text generator. You recieve a text that is a query for search a product. You need to list
    all the restritions that the user insert in query. Return a list of restritions.
    IMPORTANT:
    - If no negative aspects are provided (i.e., the negative list is empty), return an empty string ("").
    """

    with open('/content/prompts/system_prompt_generate_query_restritions_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for constraint preservation rate"
          )

No files have been modified since last commit. Skipping to prevent empty commit.


In [22]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_query_restritions_list"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    query -: \n {{user_query}} \n

    """

    with open('/content/prompts/user_prompt_generate_query_restritions_list.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for constraint preservation rate"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [32]:
print('-- generate constraint list --')
MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'
if "model" not in globals():
  model, tokenizer = load_model(MODEL_NAME)
else:
  print("Model already loaded")
generate_constraint_list(df_reviews,
                         model,tokenizer,
                         system_template_path='/content/prompts/system_prompt_generate_query_restritions_list.txt',
                         user_template_path='/content/prompts/user_prompt_generate_query_restritions_list.txt',
                         )
df_reviews_metrics = pq.read_table('/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_0.parquet')

-- generate constraint list --
Model already loaded
```plaintext
[
    "not too tingling",
    "larger size",
    "fits better"
]
```
```plaintext
great for hydration and soothing
not too tingling like some others
not too small
larger size
fits better
super hydrating and moisturizing
```

This list captures all the restrictions mentioned in the queries.
```plaintext
great for hydration and soothing
not too tingling like some others
not too small
larger size
fits better
super hydrating and moisturizing
```

Note: The last query has a repetition which might be due to formatting issues. It is considered as two separate restrictions: "not too small" and "fits better".
["doesn't require actual brow hairs for adhesion"]
["doesn't require actual brow hairs", "adhesion"]
["doesn't require actual brow hairs for adhesion"]
```plaintext
not too soft
not too tough
don't irritate daughter's skin
tough enough
not too harsh on skin or hair
too pokey
too prickly
```
```plaintext
not too soft
too tough

In [ ]:
# adjust query to return the answer as json
# resave the files from the prompts for the first notebook

In [ ]:
class ListRestritions(BaseModel):
    list_of_restritions: list[str|None]

new_rows = []
for batch in df_reviews_metrics.to_batches():
  for row in batch.to_pylist():
    negative_points = row['negative_points']
    constraint_list = row['constraint_list']
    row["consistent_score"] = 0
    for negative_point in negative_points:
      if negative_point in constraint_list:
        row['consistent_score'] += 1
      new_rows.append(row)
